In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
#importing the necessary library
import seaborn as sns
import matplotlib.pyplot as plt

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/playground-series-s4e6/sample_submission.csv
/kaggle/input/playground-series-s4e6/train.csv
/kaggle/input/playground-series-s4e6/test.csv


In [2]:
#Importing necessary libraries
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score

# Replace sparse with sparse_output
encoder = OneHotEncoder(sparse_output=True)

#loading the data frames and assigning names
df_train = pd.read_csv("/kaggle/input/playground-series-s4e6/train.csv")
df_test = pd.read_csv("/kaggle/input/playground-series-s4e6/test.csv")
df_sub =pd.read_csv("/kaggle/input/playground-series-s4e6/sample_submission.csv")
df_train.info()



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 76518 entries, 0 to 76517
Data columns (total 38 columns):
 #   Column                                          Non-Null Count  Dtype  
---  ------                                          --------------  -----  
 0   id                                              76518 non-null  int64  
 1   Marital status                                  76518 non-null  int64  
 2   Application mode                                76518 non-null  int64  
 3   Application order                               76518 non-null  int64  
 4   Course                                          76518 non-null  int64  
 5   Daytime/evening attendance                      76518 non-null  int64  
 6   Previous qualification                          76518 non-null  int64  
 7   Previous qualification (grade)                  76518 non-null  float64
 8   Nacionality                                     76518 non-null  int64  
 9   Mother's qualification                 

In [3]:
# Keep rows 1 to 3 (exclusive of row 3, i.e., rows 1 and 2)
df_reduced = df_train.iloc[1:3000]
df_reduced.info()
df_reduced

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2999 entries, 1 to 2999
Data columns (total 38 columns):
 #   Column                                          Non-Null Count  Dtype  
---  ------                                          --------------  -----  
 0   id                                              2999 non-null   int64  
 1   Marital status                                  2999 non-null   int64  
 2   Application mode                                2999 non-null   int64  
 3   Application order                               2999 non-null   int64  
 4   Course                                          2999 non-null   int64  
 5   Daytime/evening attendance                      2999 non-null   int64  
 6   Previous qualification                          2999 non-null   int64  
 7   Previous qualification (grade)                  2999 non-null   float64
 8   Nacionality                                     2999 non-null   int64  
 9   Mother's qualification                   

,id,Marital status,Application mode,Application order,Course,Daytime/evening attendance,Previous qualification,Previous qualification (grade),Nacionality,Mother's qualification,...,Curricular units 2nd sem (credited),Curricular units 2nd sem (enrolled),Curricular units 2nd sem (evaluations),Curricular units 2nd sem (approved),Curricular units 2nd sem (grade),Curricular units 2nd sem (without evaluations),Unemployment rate,Inflation rate,GDP,Target
1,1,1,17,1,9238,1,1,125.0,1,19,...,0,6,9,0,0.000000,0,11.1,0.6,2.02,Dropout
2,2,1,17,2,9254,1,1,137.0,1,3,...,0,6,0,0,0.000000,0,16.2,0.3,-0.92,Dropout
3,3,1,1,3,9500,1,1,131.0,1,19,...,0,8,11,7,12.820000,0,11.1,0.6,2.02,Enrolled
4,4,1,1,2,9500,1,1,132.0,1,19,...,0,7,12,6,12.933333,0,7.6,2.6,0.32,Graduate
5,5,1,39,1,171,1,19,133.1,1,19,...,0,0,0,0,0.000000,0,13.9,-0.3,0.79,Dropout
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2995,2995,1,1,3,9500,1,1,141.0,1,1,...,0,8,8,8,13.906250,0,15.5,2.8,-4.06,Graduate
2996,2996,1,1,1,9070,1,1,123.0,1,1,...,0,6,6,6,14.333333,0,12.7,3.7,-1.70,Graduate
2997,2997,1,1,1,171,1,1,148.0,1,3,...,0,0,0,0,0.000000,0,15.5,2.8,-4.06,Enrolled
2998,2998,1,1,4,9556,1,1,133.0,1,1,...,0,8,16,8,12.153846,0,10.8,1.4,1.74,Graduate


In [4]:
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
# Separate features and target
X = df_reduced.drop('Target',axis=1)
y = df_reduced['Target']

# Reshape y into a 2D array with one column
y_reshaped = y.values.reshape(-1, 1)
# Initialize OneHotEncoder
encoder = OneHotEncoder()

# Fit and transform on 'Category' column
one_hot_encoded = encoder.fit_transform(y_reshaped)

# Convert sparse matrix to DataFrame
one_hot_encoded_df = pd.DataFrame(one_hot_encoded.toarray(), columns=encoder.get_feature_names_out(['Target']))

# Concatenate the encoded DataFrame with the original DataFrame excluding the 'Category' column
data_encoded = pd.concat([df_reduced.drop(columns=['Target']), one_hot_encoded_df], axis=1)

In [5]:
data_encoded

,id,Marital status,Application mode,Application order,Course,Daytime/evening attendance,Previous qualification,Previous qualification (grade),Nacionality,Mother's qualification,...,Curricular units 2nd sem (evaluations),Curricular units 2nd sem (approved),Curricular units 2nd sem (grade),Curricular units 2nd sem (without evaluations),Unemployment rate,Inflation rate,GDP,Target_Dropout,Target_Enrolled,Target_Graduate
1,1.0,1.0,17.0,1.0,9238.0,1.0,1.0,125.0,1.0,19.0,...,9.0,0.0,0.000000,0.0,11.1,0.6,2.02,1.0,0.0,0.0
2,2.0,1.0,17.0,2.0,9254.0,1.0,1.0,137.0,1.0,3.0,...,0.0,0.0,0.000000,0.0,16.2,0.3,-0.92,0.0,1.0,0.0
3,3.0,1.0,1.0,3.0,9500.0,1.0,1.0,131.0,1.0,19.0,...,11.0,7.0,12.820000,0.0,11.1,0.6,2.02,0.0,0.0,1.0
4,4.0,1.0,1.0,2.0,9500.0,1.0,1.0,132.0,1.0,19.0,...,12.0,6.0,12.933333,0.0,7.6,2.6,0.32,1.0,0.0,0.0
5,5.0,1.0,39.0,1.0,171.0,1.0,19.0,133.1,1.0,19.0,...,0.0,0.0,0.000000,0.0,13.9,-0.3,0.79,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2996,2996.0,1.0,1.0,1.0,9070.0,1.0,1.0,123.0,1.0,1.0,...,6.0,6.0,14.333333,0.0,12.7,3.7,-1.70,0.0,1.0,0.0
2997,2997.0,1.0,1.0,1.0,171.0,1.0,1.0,148.0,1.0,3.0,...,0.0,0.0,0.000000,0.0,15.5,2.8,-4.06,0.0,0.0,1.0
2998,2998.0,1.0,1.0,4.0,9556.0,1.0,1.0,133.0,1.0,1.0,...,16.0,8.0,12.153846,0.0,10.8,1.4,1.74,0.0,0.0,1.0
2999,2999.0,1.0,1.0,1.0,9238.0,1.0,1.0,140.0,1.0,19.0,...,9.0,6.0,12.428571,0.0,16.2,0.3,-0.92,NaN,NaN,NaN
